In [2]:
import pandas as pd

-----

# Import each borough to produce cleaned CSV

In [263]:
df = pd.read_csv("Queens421a.csv")

In [264]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT
0,4,ARVERNE,03 3-FAMILY,1,15910,29,C0,324 BEACH 66 STREET,11692,3,0,3,"3,650","3,171",1998
1,4,ARVERNE,03 3-FAMILY,1,15910,31,C0,328 BEACH 66 STREET,11692,3,0,3,"2,217","3,171",1998
2,4,ARVERNE,03 3-FAMILY,1,15910,32,C0,330 BEACH 66 STREET,11692,3,0,3,"3,233","3,171",1998
3,4,ARVERNE,03 3-FAMILY,1,15910,57,C0,3-29 BEACH 65 STREET,11692,3,0,3,"14,270","3,000",2006
4,4,ARVERNE,03 3-FAMILY,1,15910,64,C0,3-17 BEACH 65 STREET,11692,3,0,3,"4,000","3,000",2005


In [265]:
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'BUILDING CLASS AT PRESENT',
       'ADDRESS', 'ZIP CODE', ' RESIDENTIAL UNITS ', ' COMMERCIAL UNITS ',
       ' TOTAL UNITS ', ' LAND SQUARE FEET ', ' GROSS SQUARE FEET ',
       'YEAR BUILT'],
      dtype='object')

In [266]:
# drop columns
dfDrop = df.drop(['NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 
        'BUILDING CLASS AT PRESENT',' COMMERCIAL UNITS ', ' TOTAL UNITS ', 
        ' LAND SQUARE FEET ', ' GROSS SQUARE FEET ',
       'YEAR BUILT'], axis=1)

In [267]:
dfDrop.head()

,BOROUGH,ADDRESS,ZIP CODE,RESIDENTIAL UNITS
0,4,324 BEACH 66 STREET,11692,3
1,4,328 BEACH 66 STREET,11692,3
2,4,330 BEACH 66 STREET,11692,3
3,4,3-29 BEACH 65 STREET,11692,3
4,4,3-17 BEACH 65 STREET,11692,3


In [268]:
#Sum up the number of residential units per address

unitSum = dfDrop.groupby('ADDRESS')[' RESIDENTIAL UNITS '].sum()

In [269]:
unitSum

ADDRESS
1-58 BEACH 91 STREET                          3
1-60 BEACH 101 STREET                         3
1-60 BEACH 91 STREET                          3
1-64 BEACH 101 STREET                         3
1-68 BEACH 101 STREET                         3
1-71 BEACH 91 STREET                          3
1-72 BEACH 101 STREET                         3
1-73 BEACH 91 STREET                          3
1-74 BEACH 102 STREET                         3
1-75 BEACH 91 STREET                          3
1-76 BEACH 101 STREET                         3
1-76 BEACH 102 STREET                         3
1-77 BEACH 91 STREET                          3
1-78 BEACH 102 STREET                         3
1-80 BEACH 101 STREET                         3
1-84 BEACH 101 STREET                         3
1-88 BEACH 101 STREET                         3
10-12 47 AVENUE                               7
10-17 JACKSON AVENUE                         32
10-19 NEILSON STREET                          3
10-21 NEILSON STREET            

In [270]:
#Isolate and sort unique addresses

addresses = dfDrop.ADDRESS.unique()
addresses.sort()

In [271]:
addresses

array(['1-58 BEACH 91 STREET                     ',
       '1-60 BEACH 101 STREET                    ',
       '1-60 BEACH 91 STREET                     ', ...,
       '99-47 63 ROAD                            ',
       'CONSTITUTION PLACE                       ',
       'CONSTITUTUION PLACE                      '], dtype=object)

In [272]:
#confirm that the series are of the same length. Output should be 0

len(addresses) - len(unitSum)

0

In [273]:
#Make new DF with both fields

dfNew = pd.DataFrame(columns=['address', 'unitCount'])

In [274]:
# Make list of unitSum for appending to dataframe

countList = []
for i in range(len(unitSum)):
    C = unitSum[i]
    countList.append(C)

In [275]:
# Add unique addresses to DF

dfNew['address'] = addresses
dfNew['unitCount'] = countList

In [276]:
dfNewClean = dfNew[dfNew['address'] != '                                         ']
dfClean = dfNewClean[dfNewClean['unitCount'] != 0]

In [277]:
dfClean.head()

,address,unitCount
0,1-58 BEACH 91 STREET,3
1,1-60 BEACH 101 STREET,3
2,1-60 BEACH 91 STREET,3
3,1-64 BEACH 101 STREET,3
4,1-68 BEACH 101 STREET,3


In [278]:
# Merge old dataframe info (zipcode, borough) onto new dataframe

dfMerge = dfClean.merge(dfDrop, left_on='address', right_on='ADDRESS', how='left')

In [279]:
dfMerge.head()

,address,unitCount,BOROUGH,ADDRESS,ZIP CODE,RESIDENTIAL UNITS
0,1-58 BEACH 91 STREET,3,4,1-58 BEACH 91 STREET,11693,3
1,1-60 BEACH 101 STREET,3,4,1-60 BEACH 101 STREET,11694,1
2,1-60 BEACH 101 STREET,3,4,1-60 BEACH 101 STREET,11694,1
3,1-60 BEACH 101 STREET,3,4,1-60 BEACH 101 STREET,11694,1
4,1-60 BEACH 91 STREET,3,4,1-60 BEACH 91 STREET,11693,3


In [280]:
# Drop unnecessary columns

dfFinal = dfMerge.drop(['ADDRESS', ' RESIDENTIAL UNITS '], axis=1)

In [281]:
# Drop duplicate addresses

dfFinalDrop = dfFinal.drop_duplicates('address')

In [282]:
# Final Product

dfFinalDrop.head()

,address,unitCount,BOROUGH,ZIP CODE
0,1-58 BEACH 91 STREET,3,4,11693
1,1-60 BEACH 101 STREET,3,4,11694
4,1-60 BEACH 91 STREET,3,4,11693
5,1-64 BEACH 101 STREET,3,4,11694
8,1-68 BEACH 101 STREET,3,4,11694


In [283]:
#Export to CSV
dfFinalDrop.to_csv('dfQueens', index=False)

---

# Combine individual borough CSV files into one massive file

In [3]:
dfMan = pd.read_csv('dfManhattan')
dfBX = pd.read_csv('dfBronx')
dfBK = pd.read_csv('dfBrooklyn')
dfQueens = pd.read_csv('dfQueens')
dfSI = pd.read_csv('dfStatenIsland')

In [4]:
# concat the dfs together to make final df

df421a = pd.concat([dfMan, dfBX, dfBK, dfQueens, dfSI], ignore_index=True)

In [5]:
#Confirm that concatenated dataframe is same length as each individual df

len(df421a) - (len(dfMan) + len(dfBX) + len(dfBK) + len(dfQueens) + len(dfSI))

0

In [6]:
df421a.tail()

,address,unitCount,BOROUGH,ZIP CODE
10587,801 NARROWS ROAD NORTH,79,5,10304
10588,85 MONTGOMERY AVENUE,5,5,10301
10589,89 MAPLE AVENUE,3,5,10302
10590,91 MAPLE AVENUE,3,5,10302
10591,97 VICTORY BOULEVARD,4,5,10301


In [11]:
#remove trailing spaces in address

addressStrip = []

for i in range(len(df421a)):
    address = df421a['address'][i].strip()
    addressStrip.append(address)

In [12]:
df421a.address = addressStrip

In [41]:
state = []

for i in range(len(df421a)):
    state.append("NY")

df421a.state = state

In [42]:
df421a['state'] = state

In [37]:
df421a['newBorough'] = df421a['BOROUGH'].map({1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'})

In [43]:
df421a.tail()

,address,unitCount,BOROUGH,ZIP CODE,state,newBorough
10587,801 NARROWS ROAD NORTH,79,5,10304,NY,Staten Island
10588,85 MONTGOMERY AVENUE,5,5,10301,NY,Staten Island
10589,89 MAPLE AVENUE,3,5,10302,NY,Staten Island
10590,91 MAPLE AVENUE,3,5,10302,NY,Staten Island
10591,97 VICTORY BOULEVARD,4,5,10301,NY,Staten Island


In [44]:
#Export to CSV
df421a.to_csv('421a Final w State', index=False)